In [27]:
import requests
import json
import polars as pl
import pandas as pd
import numpy as np
import time
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime

## read in production info

In [8]:
iteminfo = pl.read_json("lw_item_info_11_22_24.json") # data from mongo
iteminfo = iteminfo.to_pandas()
iteminfo = iteminfo.drop(['_id'], axis=1).copy()
iteminfo.columns = [i.upper() for i in iteminfo.columns]
iteminfo.head()

,KEYWORDS,CASTS,DIRECTORS,TITLE,TOTALRECOMMENDATIONS,TOTALDISLIKES,TOTALRATINGS,AVERAGERATING,AVERAGERATINGFORCALCS,THIRDPARTYID,THIRDPARTYPROVIDER,IMDBID,CATEGORY,PRODUCTIONCOMPANIES
0,"[police, gangster]","[Buster Keaton, Jimmy Durante, Roscoe Ates, Ph...",[Edward Sedgwick],The Gallows,1,0,1.0,NaN,NaN,0053576ec9fc4137948af2b9d246e27c,CUSTOM,tt0024762,Movies,[Metro-Goldwyn-Mayer]
1,"[videotape, murder, psychopathic-killer, psych...",[],[],The Poughkeepsie Tapes,1,0,1.0,NaN,NaN,0611f4bec47945239556c5e39ad1a703,CUSTOM,tt1010271,Movies,[]
2,"[biological weapon, philadelphia, pennsylvania...","[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],The Poughkeepsie Tapes,1,0,1.0,NaN,NaN,063a6a15e504448bab952735416df380,CUSTOM,tt0114746,Movies,"[Universal Pictures, Atlas Entertainment, Clas..."
3,"[biological weapon, philadelphia, pennsylvania...","[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],The Poughkeepsie Tapes,11,0,2.0,3.0,3.0,063a6a15e504448bab952735416df380,TMDB,tt0114746,Movies,"[Universal Pictures, Atlas Entertainment, Clas..."
4,"[mixed-marriage, gambling-debt]",[],[],Firebrand,0,0,0.0,NaN,NaN,0971712630f04584a0d9aa510bb0d27f,CUSTOM,tt15000156,Movies,[]


In [9]:
iteminfo.columns

Index(['KEYWORDS', 'CASTS', 'DIRECTORS', 'TITLE', 'TOTALRECOMMENDATIONS',
       'TOTALDISLIKES', 'TOTALRATINGS', 'AVERAGERATING',
       'AVERAGERATINGFORCALCS', 'THIRDPARTYID', 'THIRDPARTYPROVIDER', 'IMDBID',
       'CATEGORY', 'PRODUCTIONCOMPANIES'],
      dtype='object')

In [10]:
col_names = {
    'KEYWORDS': 'KEYWORDS_BI',
    'CASTS': 'CASTS_BI',
    'DIRECTORS': 'DIRECTORS_BI',
    'TITLE': 'TITLE_BI',
    'TOTALRECOMMENDATIONS': 'TOTAL_RECOMMENDATIONS_BI',
    'TOTALDISLIKES': 'TOTAL_DISLIKES_BI',
    'TOTALRATINGS': 'TOTAL_RATINGS_BI',
    'AVERAGERATING': 'AVERAGE_RATING_BI',
    'AVERAGERATINGFORCALCS': 'AVERAGE_RATING_FOR_CALCS_BI',
    'CATEGORY': 'CATEGORY_BI',
    'THIRDPARTYID': 'THIRD_PARTY_ID_BI',
    'THIRDPARTYPROVIDER': 'THIRD_PARTY_PROVIDER_BI',
    'ITEM_ID_CATEGORY': 'ITEM_ID_CATEGORY_BI',
    'IMDBID': 'IMDB_ID_BI',
    'PRODUCTIONCOMPANIES': 'PRODUCTION_COMPANIES_BI',
    'THIRD_PARTY_ID_BI': 'ITEM_ID_BI'
}

iteminfo = iteminfo.rename(columns=col_names)

In [11]:
formatted_date = datetime.today().strftime("%Y-%m-%d")
iteminfo['DATE_RETRIEVED_DATE_BI'] = formatted_date

In [12]:
iteminfo = iteminfo.rename(columns = {'THIRD_PARTY_ID_BI': 'ITEM_ID_BI'})
iteminfo['ITEM_ID_CATEGORY_BI'] = [i[0] + '_' + i[1].lower() for i in zip(iteminfo.ITEM_ID_BI, iteminfo.CATEGORY_BI)]

In [13]:
col_order = ['ITEM_ID_BI', 
             'ITEM_ID_CATEGORY_BI',
             'THIRD_PARTY_PROVIDER_BI',
             'IMDB_ID_BI', 
             'TITLE_BI', 
             'CATEGORY_BI', 
             'CASTS_BI', 
             'DIRECTORS_BI',
            'PRODUCTION_COMPANIES_BI', 
             'KEYWORDS_BI',
            'TOTAL_RECOMMENDATIONS_BI',
            'TOTAL_DISLIKES_BI',
            'TOTAL_RATINGS_BI',
            'AVERAGE_RATING_BI',
            'AVERAGE_RATING_FOR_CALCS_BI',
            'DATE_RETRIEVED_DATE_BI']

iteminfo = iteminfo[col_order]

In [14]:
iteminfo[iteminfo.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)]
# ok theres dupes here.

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI
2,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,CUSTOM,tt0114746,The Poughkeepsie Tapes,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",1,0,1.0,NaN,NaN,2025-01-03
3,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,The Poughkeepsie Tapes,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",11,0,2.0,3.0,3.000000,2025-01-03
4,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,CUSTOM,tt15000156,Firebrand,Movies,[],[],[],"[mixed-marriage, gambling-debt]",0,0,0.0,NaN,NaN,2025-01-03
5,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,TMDB,None,Firebrand,Movies,[],[],None,[],0,0,0.0,NaN,NaN,2025-01-03
23234,11977,11977_movies,TMDB,tt0080487,Caddyshack,Movies,"[Chevy Chase, Rodney Dangerfield, Ted Knight, ...",[Harold Ramis],"[Orion Pictures, Warner Bros. Pictures]","[golf course, golf, romantic rivalry, blonde, ...",49535,471,48763.0,3.9,3.898671,2025-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325464,99652,99652_shows,CUSTOM,tt9257258,KEVIN CAN F**K HIMSELF,Shows,"[Annie Murphy, Eric Petersen, Mary Hollis Inbo...",[],"[AMC Networks, Le Train Train]","[secret, boston massachusetts, political campa...",290,7,0.0,NaN,NaN,2025-01-03
325676,d0d8e6f4e2d34649ba4522075cbf6fd1,d0d8e6f4e2d34649ba4522075cbf6fd1_shows,CUSTOM,tt12327578,Star Trek: Strange New Worlds,Shows,[],[],[],"[space, alien, starship, planet, captain, futu...",0,0,0.0,NaN,NaN,2025-01-03
325677,d0d8e6f4e2d34649ba4522075cbf6fd1,d0d8e6f4e2d34649ba4522075cbf6fd1_shows,TMDB,None,Star Trek: Strange New Worlds,Shows,[],[],None,[],0,0,0.0,NaN,NaN,2025-01-03
325678,f75ff690d7414e73a9d1d994750455ba,f75ff690d7414e73a9d1d994750455ba_shows,CUSTOM,tt12809988,Sweet Tooth,Shows,[],[],[],"[frame-up, chocolate]",1,0,1.0,NaN,NaN,2025-01-03


In [15]:
iteminfo[iteminfo['ITEM_ID_CATEGORY_BI'] == '99652_shows']
# duplicates

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI
325463,99652,99652_shows,TMDB,None,Kevin Can F**K Himself,Shows,[Annie Murphy],[],[AMC Networks],"[secret, boston-massachusetts, political-campa...",1449,154,1511.0,3.8,3.778756,2025-01-03
325464,99652,99652_shows,CUSTOM,tt9257258,KEVIN CAN F**K HIMSELF,Shows,"[Annie Murphy, Eric Petersen, Mary Hollis Inbo...",[],"[AMC Networks, Le Train Train]","[secret, boston massachusetts, political campa...",290,7,0.0,NaN,NaN,2025-01-03


In [16]:
iteminfo[iteminfo['ITEM_ID_CATEGORY_BI'] == '11977_movies']
# more duplicates

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI
23234,11977,11977_movies,TMDB,tt0080487,Caddyshack,Movies,"[Chevy Chase, Rodney Dangerfield, Ted Knight, ...",[Harold Ramis],"[Orion Pictures, Warner Bros. Pictures]","[golf course, golf, romantic rivalry, blonde, ...",49535,471,48763.0,3.9,3.898671,2025-01-03
23235,11977,11977_movies,CUSTOM,tt0080487,Caddyshack,Movies,"[Chevy Chase, Rodney Dangerfield, Ted Knight, ...",[Harold Ramis],"[Orion Pictures, Warner Bros. Pictures]","[competition, lightning, golf, underwear, spor...",1835,24,0.0,NaN,NaN,2025-01-03


In [17]:
iteminfo[iteminfo['THIRD_PARTY_PROVIDER_BI'] == 'CUSTOM']

# looks like "CUSTOM" provider is the source of dupes.

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI
0,0053576ec9fc4137948af2b9d246e27c,0053576ec9fc4137948af2b9d246e27c_movies,CUSTOM,tt0024762,The Gallows,Movies,"[Buster Keaton, Jimmy Durante, Roscoe Ates, Ph...",[Edward Sedgwick],[Metro-Goldwyn-Mayer],"[police, gangster]",1,0,1.0,NaN,NaN,2025-01-03
1,0611f4bec47945239556c5e39ad1a703,0611f4bec47945239556c5e39ad1a703_movies,CUSTOM,tt1010271,The Poughkeepsie Tapes,Movies,[],[],[],"[videotape, murder, psychopathic-killer, psych...",1,0,1.0,NaN,NaN,2025-01-03
2,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,CUSTOM,tt0114746,The Poughkeepsie Tapes,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",1,0,1.0,NaN,NaN,2025-01-03
4,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,CUSTOM,tt15000156,Firebrand,Movies,[],[],[],"[mixed-marriage, gambling-debt]",0,0,0.0,NaN,NaN,2025-01-03
23235,11977,11977_movies,CUSTOM,tt0080487,Caddyshack,Movies,"[Chevy Chase, Rodney Dangerfield, Ted Knight, ...",[Harold Ramis],"[Orion Pictures, Warner Bros. Pictures]","[competition, lightning, golf, underwear, spor...",1835,24,0.0,NaN,NaN,2025-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324161,97186,97186_shows,CUSTOM,tt10166602,"Love, Victor",Shows,"[Michael Cimino, Ana Ortiz, James Martinez, Ge...",[],[20th Century Fox Television],"[gay, friendship, love, 2020s, high school, ne...",7550,13,0.0,NaN,NaN,2025-01-03
324281,97400,97400_shows,CUSTOM,tt10574558,Midnight Mass,Shows,"[Zach Gilford, Hamish Linklater, Kate Siegel, ...",[],[Intrepid Pictures],"[island, vampire, community, secret, priest, h...",194,3,0.0,NaN,NaN,2025-01-03
325464,99652,99652_shows,CUSTOM,tt9257258,KEVIN CAN F**K HIMSELF,Shows,"[Annie Murphy, Eric Petersen, Mary Hollis Inbo...",[],"[AMC Networks, Le Train Train]","[secret, boston massachusetts, political campa...",290,7,0.0,NaN,NaN,2025-01-03
325676,d0d8e6f4e2d34649ba4522075cbf6fd1,d0d8e6f4e2d34649ba4522075cbf6fd1_shows,CUSTOM,tt12327578,Star Trek: Strange New Worlds,Shows,[],[],[],"[space, alien, starship, planet, captain, futu...",0,0,0.0,NaN,NaN,2025-01-03


In [18]:

iteminfo = iteminfo[iteminfo['THIRD_PARTY_PROVIDER_BI'] != "CUSTOM"].copy()


# checking for dupes again
iteminfo[iteminfo.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)]
# there's none. perfect

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI


## fold in the imdb data

In [19]:
coop = pl.read_json("lw_imdbcooper_titles_11_22_24.json") # imdb data, dumped to json from mongo

In [20]:
coop = coop.with_columns(
    pl.col("companies").struct.field("distribution").alias("distribution_companies"),
    pl.col("companies").struct.field("production").alias("production_companies")
)

In [21]:
coop = coop.with_columns(
    pl.col("imdbRating").struct.field("rating").alias("imdb_rating"),
    pl.col("imdbRating").struct.field("numberOfVotes").alias("imdb_number_of_votes")
)

In [22]:
coop = coop.to_pandas()
coop = coop.drop(['_id', 'companies', 'imdbRating'], axis=1)

coop.head()

,titleId,countries,genres,keywords,originalTitle,titleType,year,distribution_companies,production_companies,imdb_rating,imdb_number_of_votes
0,tt0000009,[US],[Romance],"[character-name-as-title, two-word-title, nick...",Miss Jerry,movie,1894.0,None,[Alexander Black Photoplays],5.3,204.0
1,tt0000147,[US],"[Documentary, News, Sport]","[national-film-registry, first-of-its-kind, pa...",The Corbett-Fitzsimmons Fight,movie,1897.0,None,[Veriscope Company],5.3,465.0
2,tt0000502,[ES],None,[based-on-zarzuela],Bohemios,movie,1905.0,None,[Gaumont Española],4.1,15.0
3,tt0000574,[AU],"[Action, Adventure, Biography, Crime, Drama, H...","[first-of-its-kind, directorial-debut, austral...",The Story of the Kelly Gang,movie,1906.0,[The Video Cellar],"[J. and N. Tait, Johnson and Gibson]",6.0,818.0
4,tt0000591,[FR],[Drama],None,The Prodigal Son,movie,1907.0,[Pathé Frères],[Pathé Frères],4.4,20.0


In [23]:
coop.columns = [i.upper() for i in coop.columns]

In [24]:
rename_cols = {
    'TITLEID': 'IMDB_ID_BI',
    'COUNTRIES': 'COUNTRIES_BI',
    'GENRES': 'GENRES_BI',
    'KEYWORDS': 'KEYWORDS2_BI',
    'ORIGINALTITLE': 'ORIGINAL_TITLE_BI',
    'TITLETYPE': 'TITLE_TYPE_BI',
    'YEAR': 'YEAR_BI',
    'DISTRIBUTION_COMPANIES': 'DISTRIBUTION_COMPANIES_BI',
    'PRODUCTION_COMPANIES': 'PRODUCTION_COMPANIES2_BI',
    'IMDB_RATING': 'IMDB_RATING_BI',
    'IMDB_NUMBER_OF_VOTES': 'IMDB_NUMBER_OF_VOTES_BI'
}

coop = coop.rename(columns=rename_cols)

In [25]:
merge1 = pd.merge(iteminfo, coop, how='left', left_on='IMDB_ID_BI', right_on='IMDB_ID_BI')
merge1.head()
# The Poughkeepsie Tapes and 12 Monkeys mismatch, row index 0 

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,...,COUNTRIES_BI,GENRES_BI,KEYWORDS2_BI,ORIGINAL_TITLE_BI,TITLE_TYPE_BI,YEAR_BI,DISTRIBUTION_COMPANIES_BI,PRODUCTION_COMPANIES2_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI
0,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,The Poughkeepsie Tapes,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",...,[US],"[Mystery, Sci-Fi, Thriller]","[male-time-traveler, brunette-woman, white-coa...",12 Monkeys,movie,1995.0,"[Universal Studios, Ascot Elite Entertainment ...","[Universal Pictures, Atlas Entertainment, Clas...",8.0,624917.0
1,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,TMDB,None,Firebrand,Movies,[],[],None,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,100_movies,TMDB,tt0120735,"Lock, Stock and Two Smoking Barrels",Movies,"[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",[Guy Ritchie],"[Summit Entertainment, The Steve Tisch Company...","[gangster, violence, united kingdom, money, ga...",...,[GB],"[Action, Comedy, Crime]","[low-budget-film, dark-comedy, violence, unite...","Lock, Stock and Two Smoking Barrels",movie,1998.0,"[BV-film AS, Becker Entertainment, Columbia Tr...","[Summit Entertainment, The Steve Tisch Company...",8.2,589412.0
3,10000,10000_movies,TMDB,tt0109747,The Strategy of the Snail,Movies,"[Fausto Cabrera, Frank Ramírez, Delfina Guido,...",[Sergio Cabrera],[Ministère de la Culture et de la Francophonie...,"[anarchism, police officer, graffiti, city, gr...",...,"[CO, IT, FR]","[Comedy, Drama]","[fissure, police-officer, graffiti, stain-on-a...",La estrategia del caracol,movie,1993.0,"[Argentina Video Home, Cinemussy, trigon-film]","[Caracol Televisión, Compañía de Fomento Cinem...",7.7,2642.0
4,1000003,1000003_movies,TMDB,tt12298986,Graham Kay: Stupid Jokes,Movies,[Graham Kay],[Mathieu Baer],[Just For Laughs Television],[stand-up comedy],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
merge1[merge1.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)]
# no dupes after the merge

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,...,COUNTRIES_BI,GENRES_BI,KEYWORDS2_BI,ORIGINAL_TITLE_BI,TITLE_TYPE_BI,YEAR_BI,DISTRIBUTION_COMPANIES_BI,PRODUCTION_COMPANIES2_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI


In [28]:
# merge1[merge1['TITLE_BI'] == '12 Monkeys']
# how many mismatches are there?
merge1[
    merge1['ORIGINAL_TITLE_BI'].notnull() & 
    (merge1['ORIGINAL_TITLE_BI'] != merge1['TITLE_BI']) & 
    merge1['COUNTRIES_BI'].apply(lambda x: 'US' in x if isinstance(x, np.ndarray) else False)
]

# nearly 30k mismatches for all countries
# 5,400 for US countries

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,...,COUNTRIES_BI,GENRES_BI,KEYWORDS2_BI,ORIGINAL_TITLE_BI,TITLE_TYPE_BI,YEAR_BI,DISTRIBUTION_COMPANIES_BI,PRODUCTION_COMPANIES2_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI
0,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,The Poughkeepsie Tapes,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",...,[US],"[Mystery, Sci-Fi, Thriller]","[male-time-traveler, brunette-woman, white-coa...",12 Monkeys,movie,1995.0,"[Universal Studios, Ascot Elite Entertainment ...","[Universal Pictures, Atlas Entertainment, Clas...",8.0,624917.0
40,1000283,1000283_movies,TMDB,tt11605552,"Tethered, Are We the Experiment?",Movies,"[Dr. Dimitri Christakis MD, MPH, Jessica Wong,...",[Daniel Gartzke],[],[],...,[US],[Documentary],None,Tethered,movie,2021.0,"[Conscious Content LLC, Filmhub]","[Conscious Content LLC, Shaman Motion Pictures]",NaN,NaN
55,1000368,1000368_movies,TMDB,tt4332600,"Elmore Leonard: ""But Don't Try to Write""",Movies,"[Campbell Scott, Wendy Calhoun, Bridgett M. Da...",[John Mulholland],"[Transmultimedia Entertainment, Dutch Films, L...","[biography, writer, crime fiction writer, docu...",...,[US],[Documentary],None,Elmore Leonard - But Don't Try to Write,movie,2021.0,"[American Public Television Worldwide, Transmu...",[Transmultimedia Entertainment],9.4,14.0
66,1000475,1000475_movies,TMDB,tt21291992,River Wild,Movies,"[Adam Brody, Leighton Meester, Taran Killam, O...",[Ben Ketai],"[Universal 1440 Entertainment, Hero Squared]","[friend, love]",...,[US],[Thriller],None,The River Wild,movie,NaN,None,"[Universal 1440 Entertainment, Hero Squared]",NaN,NaN
80,1000553,1000553_movies,TMDB,tt7889878,We Bought A Haunted House,Movies,[],"[Ray Etheridge, Migdalia Etheridge]",[Golden West Films],[],...,[US],[Mystery],None,We Bought a Haunted House,movie,2018.0,[Golden West Films],[Golden West Films],2.5,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324932,98963,98963_shows,TMDB,tt2288831,Hollywood Girls,Shows,[Nabilla Benattia],[],"[400 Blows Production, La Grosse Equipe, NRJ 12]",[],...,"[FR, US]",[Drama],None,Hollywood Girls 2,tvSeries,2012.0,[NRJ12],"[400 Blows Production, La Grosse Equipe, NRJ12]",4.5,23.0
325234,9946,9946_shows,TMDB,tt0078586,Co-Ed Fever,Shows,"[Heather Thomas, Alexa Kenin, David Keith, Ham...",[],[],"[college, teenage girl]",...,[US],[Comedy],"[college, teenage-girl, canceled-after-one-epi...",Co-ed Fever,tvSeries,1979.0,[CBS],None,5.1,45.0
325273,99511,99511_shows,TMDB,tt3076808,America the Wild with Casey Anderson,Shows,[Casey Anderson],[],[],[mountain],...,[US],[Reality-TV],"[mountain, non-fiction]",America the Wild,tvSeries,2013.0,[National Geographic Channel],[Grizzly Peak Films],8.3,29.0
325425,9977,9977_shows,TMDB,tt0040028,Arthur Godfrey's Talent Scouts,Shows,[],[],[],"[1950s, 1940s, competition]",...,[US],"[Comedy, Family, Music]","[talent, talent-contest, 1950s, 1940s, competi...",Talent Scouts,tvSeries,1948.0,"[CBS, Vioobu]",[CBS],6.7,23.0


In [29]:
# Dealing with the title mismatch

# I wonder if original title is just the better thing to use to the title mismatch.
# So when original title is there I use original title else title
merge1['TITLE_BI'] = np.where(
    merge1['ORIGINAL_TITLE_BI'].notnull(),  # Condition: ORIGINAL_TITLE_BI is not null
    merge1['ORIGINAL_TITLE_BI'],           # Value if True
    merge1['TITLE_BI']                     # Value if False
)

In [30]:
merge1.head()

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,CASTS_BI,DIRECTORS_BI,PRODUCTION_COMPANIES_BI,KEYWORDS_BI,...,COUNTRIES_BI,GENRES_BI,KEYWORDS2_BI,ORIGINAL_TITLE_BI,TITLE_TYPE_BI,YEAR_BI,DISTRIBUTION_COMPANIES_BI,PRODUCTION_COMPANIES2_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI
0,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,12 Monkeys,Movies,"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",[Terry Gilliam],"[Universal Pictures, Atlas Entertainment, Clas...","[biological weapon, philadelphia, pennsylvania...",...,[US],"[Mystery, Sci-Fi, Thriller]","[male-time-traveler, brunette-woman, white-coa...",12 Monkeys,movie,1995.0,"[Universal Studios, Ascot Elite Entertainment ...","[Universal Pictures, Atlas Entertainment, Clas...",8.0,624917.0
1,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,TMDB,None,Firebrand,Movies,[],[],None,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,100_movies,TMDB,tt0120735,"Lock, Stock and Two Smoking Barrels",Movies,"[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",[Guy Ritchie],"[Summit Entertainment, The Steve Tisch Company...","[gangster, violence, united kingdom, money, ga...",...,[GB],"[Action, Comedy, Crime]","[low-budget-film, dark-comedy, violence, unite...","Lock, Stock and Two Smoking Barrels",movie,1998.0,"[BV-film AS, Becker Entertainment, Columbia Tr...","[Summit Entertainment, The Steve Tisch Company...",8.2,589412.0
3,10000,10000_movies,TMDB,tt0109747,La estrategia del caracol,Movies,"[Fausto Cabrera, Frank Ramírez, Delfina Guido,...",[Sergio Cabrera],[Ministère de la Culture et de la Francophonie...,"[anarchism, police officer, graffiti, city, gr...",...,"[CO, IT, FR]","[Comedy, Drama]","[fissure, police-officer, graffiti, stain-on-a...",La estrategia del caracol,movie,1993.0,"[Argentina Video Home, Cinemussy, trigon-film]","[Caracol Televisión, Compañía de Fomento Cinem...",7.7,2642.0
4,1000003,1000003_movies,TMDB,tt12298986,Graham Kay: Stupid Jokes,Movies,[Graham Kay],[Mathieu Baer],[Just For Laughs Television],[stand-up comedy],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
def merge_and_unique_lists(tup):
    try:
        list1 = list(tup[0])
        list2 = list(tup[1])
        return list({item.lower() for item in list1 + list2})
    except Exception as e:
        return []

In [32]:
merge1['PRODUCTION_COMPANIES_BI'] = [[] if isinstance(i, float) else i for i in merge1.PRODUCTION_COMPANIES_BI]
merge1['KEYWORDS_BI'] = [[] if isinstance(i, float) else i for i in merge1.KEYWORDS_BI]
merge1['PRODUCTION_COMPANIES2_BI'] = [[] if isinstance(i, float) else i for i in merge1.PRODUCTION_COMPANIES2_BI]
merge1['KEYWORDS2_BI'] = [[] if isinstance(i, float) else i for i in merge1.KEYWORDS2_BI]

merge1['PRODUCTION_COMPANIES_BI'] = [merge_and_unique_lists(i) for i in zip(merge1.PRODUCTION_COMPANIES_BI, merge1.PRODUCTION_COMPANIES2_BI)]
merge1['KEYWORDS_BI'] = [merge_and_unique_lists(i) for i in zip(merge1.KEYWORDS_BI, merge1.KEYWORDS2_BI)]


In [33]:
drop_these = ['ORIGINAL_TITLE_BI', 'KEYWORDS2_BI', 'TITLE_TYPE_BI', 'PRODUCTION_COMPANIES2_BI']

merge1 = merge1.drop(drop_these, axis=1)

In [34]:
column_order = ['ITEM_ID_BI', 
                'ITEM_ID_CATEGORY_BI', 
                'THIRD_PARTY_PROVIDER_BI',
                'IMDB_ID_BI', 
                'TITLE_BI', 
                'CATEGORY_BI', 
                'GENRES_BI', # not in
                'YEAR_BI', # not in
                'COUNTRIES_BI', # not in
                'CASTS_BI', 
                'DIRECTORS_BI',
                'PRODUCTION_COMPANIES_BI', 
                'DISTRIBUTION_COMPANIES_BI',
                'KEYWORDS_BI', 
                'TOTAL_RECOMMENDATIONS_BI',
                'TOTAL_DISLIKES_BI', 
                'TOTAL_RATINGS_BI', 
                'AVERAGE_RATING_BI',
                'AVERAGE_RATING_FOR_CALCS_BI', 
                'DATE_RETRIEVED_DATE_BI',
                'IMDB_RATING_BI',
                'IMDB_NUMBER_OF_VOTES_BI']

merge1 = merge1[column_order].copy()

In [35]:
merge1.head()
# 12 Monkeys solved.

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,GENRES_BI,YEAR_BI,COUNTRIES_BI,CASTS_BI,...,DISTRIBUTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI
0,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,12 Monkeys,Movies,"[Mystery, Sci-Fi, Thriller]",1995.0,[US],"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",...,"[Universal Studios, Ascot Elite Entertainment ...","[beaten-to-death, microbiologist, airplane, re...",11,0,2.0,3.0,3.000000,2025-01-03,8.0,624917.0
1,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,TMDB,None,Firebrand,Movies,NaN,NaN,NaN,[],...,NaN,[],0,0,0.0,NaN,NaN,2025-01-03,NaN,NaN
2,100,100_movies,TMDB,tt0120735,"Lock, Stock and Two Smoking Barrels",Movies,"[Action, Comedy, Crime]",1998.0,[GB],"[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",...,"[BV-film AS, Becker Entertainment, Columbia Tr...","[beaten-to-death, con-man, snorricam, drink-th...",2001,17,1983.0,3.9,3.906556,2025-01-03,8.2,589412.0
3,10000,10000_movies,TMDB,tt0109747,La estrategia del caracol,Movies,"[Comedy, Drama]",1993.0,"[CO, IT, FR]","[Fausto Cabrera, Frank Ramírez, Delfina Guido,...",...,"[Argentina Video Home, Cinemussy, trigon-film]","[transgender, gun-shooting, revenge, fissure, ...",2,0,2.0,NaN,NaN,2025-01-03,7.7,2642.0
4,1000003,1000003_movies,TMDB,tt12298986,Graham Kay: Stupid Jokes,Movies,NaN,NaN,NaN,[Graham Kay],...,NaN,[stand-up comedy],0,0,0.0,NaN,NaN,2025-01-03,NaN,NaN


In [36]:
merge1[merge1.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)]
# no dupes after processing merge1. Now on the merging the TMDB data

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,GENRES_BI,YEAR_BI,COUNTRIES_BI,CASTS_BI,...,DISTRIBUTION_COMPANIES_BI,KEYWORDS_BI,TOTAL_RECOMMENDATIONS_BI,TOTAL_DISLIKES_BI,TOTAL_RATINGS_BI,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,DATE_RETRIEVED_DATE_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI


## now on to the tmdb data

In [43]:
file_path = 'tmdb_show_data_11_21_24.json'
tmdb = pd.read_json(file_path)

In [44]:
cols_to_keep = ["created_by", 
                   "id", 
                   "name", 
                   "networks",
                 "genres",
                   "origin_country", 
                   "original_name", 
                  "production_companies",
                   "first_air_date",
                   "last_air_date",
                  "type"]

tmdb = tmdb[cols_to_keep].copy()

In [45]:
def get_value(lst, key_name):
    try:
        return [dikt['name'] for dikt in lst]
    except Exception as e:
        return []

In [46]:
tmdb['CREATORS'] = [get_value(i, 'name') for i in tmdb.created_by]
tmdb['ORIGINAL_NETWORKS'] = [get_value(i, 'name') for i in tmdb.networks]
tmdb['PRODUCTION_COMPANIES'] = [get_value(i, 'name') for i in tmdb.production_companies]
tmdb['GENRES'] = [get_value(i, 'name') for i in tmdb.genres]

In [47]:
tmdb = tmdb.drop(['created_by', 'networks', 'production_companies'], axis=1).copy()
tmdb.head()

,id,name,genres,origin_country,original_name,first_air_date,last_air_date,type,CREATORS,ORIGINAL_NETWORKS,PRODUCTION_COMPANIES,GENRES
0,66732.0,Stranger Things,"[{'id': 18, 'name': 'Drama'}, {'id': 10765, 'n...",[US],Stranger Things,2016-07-15,2022-07-01,Scripted,"[Ross Duffer, Matt Duffer]",[Netflix],"[21 Laps Entertainment, Monkey Massacre Produc...","[Drama, Sci-Fi & Fantasy, Mystery, Action & Ad..."
1,2316.0,The Office,"[{'id': 35, 'name': 'Comedy'}]",[US],The Office,2005-03-24,2013-05-16,Scripted,[Greg Daniels],[NBC],"[Universal Television, Deedle-Dee Productions,...",[Comedy]
2,1416.0,Grey's Anatomy,"[{'id': 18, 'name': 'Drama'}]",[US],Grey's Anatomy,2005-03-27,2024-11-14,Scripted,[Shonda Rhimes],[ABC],"[The Mark Gordon Company, shondaland, ABC Stud...",[Drama]
3,18165.0,The Vampire Diaries,"[{'id': 18, 'name': 'Drama'}, {'id': 10765, 'n...",[US],The Vampire Diaries,2009-09-10,2017-03-10,Scripted,"[Kevin Williamson, Julie Plec]",[The CW],"[Warner Bros. Television, Alloy Entertainment,...","[Drama, Sci-Fi & Fantasy]"
4,1434.0,Family Guy,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",[US],Family Guy,1999-01-31,2024-04-17,Scripted,[Seth MacFarlane],[FOX],"[20th Century Fox Television, Fuzzy Door Produ...","[Animation, Comedy]"


In [48]:
tmdb = tmdb.drop(['genres'], axis=1).copy()
tmdb = tmdb.dropna(subset=['id'])
tmdb['ITEM_ID'] = [str(int(i)) for i in tmdb.id]
tmdb = tmdb.drop(['id'], axis=1).copy()
tmdb.columns = [i.upper() for i in tmdb.columns]
tmdb = tmdb.drop(['NAME'], axis=1).copy()
tmdb['ITEM_ID_CATEGORY_BI'] = [i + '_shows' for i in tmdb.ITEM_ID]
tmdb.head()

,ORIGIN_COUNTRY,ORIGINAL_NAME,FIRST_AIR_DATE,LAST_AIR_DATE,TYPE,CREATORS,ORIGINAL_NETWORKS,PRODUCTION_COMPANIES,GENRES,ITEM_ID,ITEM_ID_CATEGORY_BI
0,[US],Stranger Things,2016-07-15,2022-07-01,Scripted,"[Ross Duffer, Matt Duffer]",[Netflix],"[21 Laps Entertainment, Monkey Massacre Produc...","[Drama, Sci-Fi & Fantasy, Mystery, Action & Ad...",66732,66732_shows
1,[US],The Office,2005-03-24,2013-05-16,Scripted,[Greg Daniels],[NBC],"[Universal Television, Deedle-Dee Productions,...",[Comedy],2316,2316_shows
2,[US],Grey's Anatomy,2005-03-27,2024-11-14,Scripted,[Shonda Rhimes],[ABC],"[The Mark Gordon Company, shondaland, ABC Stud...",[Drama],1416,1416_shows
3,[US],The Vampire Diaries,2009-09-10,2017-03-10,Scripted,"[Kevin Williamson, Julie Plec]",[The CW],"[Warner Bros. Television, Alloy Entertainment,...","[Drama, Sci-Fi & Fantasy]",18165,18165_shows
4,[US],Family Guy,1999-01-31,2024-04-17,Scripted,[Seth MacFarlane],[FOX],"[20th Century Fox Television, Fuzzy Door Produ...","[Animation, Comedy]",1434,1434_shows


In [49]:
# make lists into tuples in order to drop dupes
tmdb = tmdb.applymap(lambda x: tuple(x) if isinstance(x, list) else x)

tmdb = tmdb.drop_duplicates()

tmdb = tmdb.applymap(lambda x: list(x) if isinstance(x, tuple) else x)

In [50]:
tmdb[tmdb.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)] # no dupes in tmdb

,ORIGIN_COUNTRY,ORIGINAL_NAME,FIRST_AIR_DATE,LAST_AIR_DATE,TYPE,CREATORS,ORIGINAL_NETWORKS,PRODUCTION_COMPANIES,GENRES,ITEM_ID,ITEM_ID_CATEGORY_BI


## merging

In [51]:
merge2 = pd.merge(merge1, tmdb, how='left', left_on='ITEM_ID_CATEGORY_BI', right_on='ITEM_ID_CATEGORY_BI')

In [52]:
merge2[merge2.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)] # no dupes in merge2

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,GENRES_BI,YEAR_BI,COUNTRIES_BI,CASTS_BI,...,ORIGIN_COUNTRY,ORIGINAL_NAME,FIRST_AIR_DATE,LAST_AIR_DATE,TYPE,CREATORS,ORIGINAL_NETWORKS,PRODUCTION_COMPANIES,GENRES,ITEM_ID


In [53]:
drop_these = ['ORIGINAL_NAME', 'ITEM_ID']
merge2 = merge2.drop(drop_these, axis=1).copy()
merge2 = merge2.rename(columns = {'TYPE': 'SHOW_TYPE'}).copy()

In [54]:
def merge_and_unique_lists(tup):    
    try:
        list1 = list(tup[0])
        list2 = list(tup[1])
        return list({item.lower() for item in list1 + list2})
    except Exception as e:
        return []
    
def merge_and_unique_lists_upper(tup):
    try:
        list1 = list(tup[0])
        list2 = list(tup[1])
        return list({item.upper() for item in list1 + list2})
    except Exception as e:
        return []

In [55]:
#cleaning data types
merge2['GENRES_BI'] = [[] if isinstance(i, float) else i for i in merge2.GENRES_BI]
merge2['PRODUCTION_COMPANIES_BI'] = [[] if isinstance(i, float) else i for i in merge2.PRODUCTION_COMPANIES_BI]
merge2['COUNTRIES_BI'] = [[] if isinstance(i, float) else i for i in merge2.COUNTRIES_BI]

merge2['GENRES'] = [[] if isinstance(i, float) else i for i in merge2.GENRES]
merge2['PRODUCTION_COMPANIES'] = [[] if isinstance(i, float) else i for i in merge2.PRODUCTION_COMPANIES]
merge2['ORIGIN_COUNTRY'] = [[] if isinstance(i, float) else i for i in merge2.ORIGIN_COUNTRY]

In [56]:
merge2['PRODUCTION_COMPANIES_BI'][0]

['twelve monkeys productions',
 'atlas entertainment',
 'universal pictures',
 'classico']

In [57]:
merge2['PRODUCTION_COMPANIES_BI'] = [merge_and_unique_lists(i) for i in zip(merge2.PRODUCTION_COMPANIES_BI, merge2.PRODUCTION_COMPANIES)]
merge2['COUNTRIES_BI'] = [merge_and_unique_lists_upper(i) for i in zip(merge2.COUNTRIES_BI, merge2.ORIGIN_COUNTRY)]
merge2['GENRES_BI'] = [merge_and_unique_lists(i) for i in zip(merge2.GENRES_BI, merge2.GENRES)]


In [58]:
merge2[merge2['TITLE_BI'] == 'Stranger Things']['COUNTRIES_BI'] # ok, empty lists where needed

4161            []
8950            []
10957           []
29990           []
48427     [US, GB]
170625          []
207350        [US]
223154          []
223432          []
227960          []
232864          []
241834          []
242851          []
242981          []
243976          []
244194          []
244626          []
248915          []
282066          []
305770        [US]
320439          []
Name: COUNTRIES_BI, dtype: object

In [59]:
merge2 = merge2.drop(['ORIGIN_COUNTRY', 'PRODUCTION_COMPANIES', 'GENRES'], axis=1).copy()

In [60]:
rename_these = {
    'FIRST_AIR_DATE': 'FIRST_AIR_DATE_BI',
    'LAST_AIR_DATE': 'LAST_AIR_DATE_BI',
    'SHOW_TYPE': 'SHOW_TYPE_BI',
    'CREATORS': 'CREATORS_BI',
    'ORIGINAL_NETWORKS': 'ORIGINAL_NETWORKS_BI'
}

merge2 = merge2.rename(columns=rename_these).copy()

In [61]:
col_order = ['ITEM_ID_BI', 
             'ITEM_ID_CATEGORY_BI', 
             'THIRD_PARTY_PROVIDER_BI',
             'IMDB_ID_BI', 
             'TITLE_BI', 
             'CATEGORY_BI', 
             'GENRES_BI', 
             'YEAR_BI',
             'COUNTRIES_BI',
             'CASTS_BI', 
             'DIRECTORS_BI', 
             'PRODUCTION_COMPANIES_BI',
             'DISTRIBUTION_COMPANIES_BI', 
             'KEYWORDS_BI', 
             'TOTAL_RECOMMENDATIONS_BI',
             'TOTAL_DISLIKES_BI', 
             'TOTAL_RATINGS_BI', 
             'AVERAGE_RATING_BI',
             'AVERAGE_RATING_FOR_CALCS_BI',
             'FIRST_AIR_DATE_BI',
             'LAST_AIR_DATE_BI', 
             'SHOW_TYPE_BI', 
             'CREATORS_BI',
             'ORIGINAL_NETWORKS_BI',
             'IMDB_RATING_BI', 
             'IMDB_NUMBER_OF_VOTES_BI',
             'DATE_RETRIEVED_DATE_BI']

merge2 = merge2[col_order].copy()

In [62]:
merge2['YEAR_BI'] = merge2['YEAR_BI'].astype(pd.Int64Dtype())

In [63]:
merge2.head()

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,GENRES_BI,YEAR_BI,COUNTRIES_BI,CASTS_BI,...,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,FIRST_AIR_DATE_BI,LAST_AIR_DATE_BI,SHOW_TYPE_BI,CREATORS_BI,ORIGINAL_NETWORKS_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI,DATE_RETRIEVED_DATE_BI
0,063a6a15e504448bab952735416df380,063a6a15e504448bab952735416df380_movies,TMDB,tt0114746,12 Monkeys,Movies,"[thriller, sci-fi, mystery]",1995,[US],"[Bruce Willis, Madeleine Stowe, Brad Pitt, Chr...",...,3.0,3.000000,NaN,NaN,NaN,NaN,NaN,8.0,624917.0,2025-01-03
1,0971712630f04584a0d9aa510bb0d27f,0971712630f04584a0d9aa510bb0d27f_movies,TMDB,None,Firebrand,Movies,[],<NA>,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-03
2,100,100_movies,TMDB,tt0120735,"Lock, Stock and Two Smoking Barrels",Movies,"[crime, comedy, action]",1998,[GB],"[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",...,3.9,3.906556,NaN,NaN,NaN,NaN,NaN,8.2,589412.0,2025-01-03
3,10000,10000_movies,TMDB,tt0109747,La estrategia del caracol,Movies,"[comedy, drama]",1993,"[CO, IT, FR]","[Fausto Cabrera, Frank Ramírez, Delfina Guido,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,2642.0,2025-01-03
4,1000003,1000003_movies,TMDB,tt12298986,Graham Kay: Stupid Jokes,Movies,[],<NA>,[],[Graham Kay],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-03


In [64]:
merge2[merge2.duplicated(subset=['ITEM_ID_CATEGORY_BI'], keep=False)] # no dupes in merge2

,ITEM_ID_BI,ITEM_ID_CATEGORY_BI,THIRD_PARTY_PROVIDER_BI,IMDB_ID_BI,TITLE_BI,CATEGORY_BI,GENRES_BI,YEAR_BI,COUNTRIES_BI,CASTS_BI,...,AVERAGE_RATING_BI,AVERAGE_RATING_FOR_CALCS_BI,FIRST_AIR_DATE_BI,LAST_AIR_DATE_BI,SHOW_TYPE_BI,CREATORS_BI,ORIGINAL_NETWORKS_BI,IMDB_RATING_BI,IMDB_NUMBER_OF_VOTES_BI,DATE_RETRIEVED_DATE_BI


### push to snowflake